In [3]:
import pandas as pd
from pathlib import Path

# ---------------------------
# 0. 파일 경로 설정
# ---------------------------
# 너의 로컬 경로에 맞게 수정해서 쓰면 됨
COHORT_PATH = Path("./cohort/cohort_ver50_only_subject_id.csv")   # subject_id만 있는 파일
CHARTEVENTS_PATH = Path("../../data/MIMIC4-hosp-icu/chartevents.csv")  # MIMIC-IV ICU chartevents
OUT_PATH = Path("../../data/MIMIC4-hosp-icu/cohort_ver50_chartevents.csv")         # 출력 파일

# ---------------------------
# 1. cohort subject_id 로딩
# ---------------------------
df_sub = pd.read_csv(COHORT_PATH)

# subject_id 컬럼 이름이 다르면 여기 수정
subject_ids = set(df_sub["subject_id"].unique())

print(f"[INFO] cohort subject_id 개수: {len(subject_ids)}")

# ---------------------------
# 2. chartevents에서 HR / BP 추출
# ---------------------------
HR_ITEMID = 220045   # Heart Rate
MBP_ITEMID = 220052  # Mean Blood Pressure (Non-invasive)

usecols = [
    "subject_id",
    "hadm_id",
    "stay_id",
    "charttime",
    "itemid",
    "valuenum",
]

# 메모리 절약을 위해 chunk 단위로 읽기
chunksize = 1_000_000

filtered_chunks = []

for i, chunk in enumerate(
    pd.read_csv(
        CHARTEVENTS_PATH,
        usecols=usecols,
        chunksize=chunksize,
        dtype={
            "subject_id": "int32",
            "hadm_id": "Int32",
            "stay_id": "Int32",
            "itemid": "int32",
            "valuenum": "float32",
        }
    )
):
    # 1) cohort subject_id만 남기기
    chunk = chunk[chunk["subject_id"].isin(subject_ids)]
    if chunk.empty:
        continue

    # 2) HR / Mean BP itemid만 남기기
    chunk = chunk[chunk["itemid"].isin([HR_ITEMID, MBP_ITEMID])]
    if chunk.empty:
        continue

    # 3) 유효 값만
    chunk = chunk[chunk["valuenum"].notna()]
    if chunk.empty:
        continue

    # 4) itemid를 변수 이름으로 매핑
    chunk["variable_name"] = chunk["itemid"].map({
        HR_ITEMID: "heart_rate",
        MBP_ITEMID: "mean_bp",
    })

    filtered_chunks.append(chunk)

    print(f"[INFO] chunk {i} 처리 완료, 현재 누적 rows: {sum(len(c) for c in filtered_chunks)}")

# ---------------------------
# 3. 결과 병합 및 저장
# ---------------------------
if filtered_chunks:
    df_vitals = pd.concat(filtered_chunks, ignore_index=True)
    # 정렬 (선택)
    df_vitals = df_vitals.sort_values(["subject_id", "hadm_id", "charttime"])

    df_vitals.to_csv(OUT_PATH, index=False)
    print(f"[DONE] 최종 vitals 저장 완료: {OUT_PATH} (rows={len(df_vitals)})")
else:
    print("[WARN] 필터에 걸리는 HR/BP 레코드가 없습니다.")

[INFO] cohort subject_id 개수: 1878
[INFO] chunk 0 처리 완료, 현재 누적 rows: 235
[INFO] chunk 2 처리 완료, 현재 누적 rows: 395
[INFO] chunk 3 처리 완료, 현재 누적 rows: 1308
[INFO] chunk 5 처리 완료, 현재 누적 rows: 1627
[INFO] chunk 7 처리 완료, 현재 누적 rows: 2061
[INFO] chunk 8 처리 완료, 현재 누적 rows: 2399
[INFO] chunk 10 처리 완료, 현재 누적 rows: 3004
[INFO] chunk 11 처리 완료, 현재 누적 rows: 5038
[INFO] chunk 12 처리 완료, 현재 누적 rows: 5937
[INFO] chunk 13 처리 완료, 현재 누적 rows: 5952
[INFO] chunk 14 처리 완료, 현재 누적 rows: 6529
[INFO] chunk 15 처리 완료, 현재 누적 rows: 6856
[INFO] chunk 16 처리 완료, 현재 누적 rows: 6998
[INFO] chunk 17 처리 완료, 현재 누적 rows: 7639
[INFO] chunk 18 처리 완료, 현재 누적 rows: 7804
[INFO] chunk 19 처리 완료, 현재 누적 rows: 8190
[INFO] chunk 20 처리 완료, 현재 누적 rows: 8246
[INFO] chunk 21 처리 완료, 현재 누적 rows: 9510
[INFO] chunk 22 처리 완료, 현재 누적 rows: 10244
[INFO] chunk 23 처리 완료, 현재 누적 rows: 12308
[INFO] chunk 24 처리 완료, 현재 누적 rows: 12459
[INFO] chunk 25 처리 완료, 현재 누적 rows: 12630
[INFO] chunk 26 처리 완료, 현재 누적 rows: 13853
[INFO] chunk 27 처리 완료, 현재 누적 rows: 14216
[INFO] c